In [ ]:
%%writefile Sources/cub-perf-sync.cu
#include "dli.h"

float simulate(int width,
               int height,
               const thrust::device_vector<float> &in,
                     thrust::device_vector<float> &out,
               bool use_cub) 
{
  cuda::std::mdspan temp_in(thrust::raw_pointer_cast(in.data()), height, width);
  auto compute = [=] __host__ __device__(int id) {
    const int column = id % width;
    const int row    = id / width;

    // loop over all points in domain (except boundary)
    if (row > 0 && column > 0 && row < height - 1 && column < width - 1)
    {
      // evaluate derivatives
      float d2tdx2 = temp_in(row, column - 1) - 2 * temp_in(row, column) + temp_in(row, column + 1);
      float d2tdy2 = temp_in(row - 1, column) - 2 * temp_in(row, column) + temp_in(row + 1, column);

      // update temperatures
      return temp_in(row, column) + 0.2f * (d2tdx2 + d2tdy2);
    }
    else
    {
      return temp_in(row, column);
    }
  };

  auto begin = std::chrono::high_resolution_clock::now();

  if (use_cub) 
  {
    // auto cell_ids = thrust::make_counting_iterator(0);
    // cub::DeviceTransform::Transform(cell_ids, out.begin(), width * height, compute);
    auto cell_ids = thrust::make_counting_iterator(0);
    cub::DeviceTransform::Transform(cell_ids, out.begin(), width * height, compute);
    cudaDeviceSynchronize();
  }
  else 
  {
    thrust::tabulate(thrust::device, out.begin(), out.end(), compute);
  }
  auto end = std::chrono::high_resolution_clock::now();
  return std::chrono::duration<float>(end - begin).count();
}

int main()
{
  std::cout << "size, thrust, cub\n";
  for (int size = 1024; size <= 16384; size *= 2)
  {
    int width = size;
    int height = size;
    thrust::device_vector<float> current_temp(height * width, 15.0f);
    thrust::device_vector<float> next_temp(height * width);

    std::cout << size << ", "
              << simulate(width, height, current_temp, next_temp, false) << ", "
              << simulate(width, height, current_temp, next_temp, true) << "\n";
  }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/cub-perf-sync.cu # build executable
!/tmp/a.out # run executable

In [ ]:
size, thrust, cub
1024, 6.8413e-05, 6.1329e-05
2048, 0.000145923, 0.00014446
4096, 0.000529286, 0.000528804
8192, 0.00207851, 0.00206931
16384, 0.00842378, 0.00803759

In [ ]:
%%writefile Sources/compute-io-overlap.cu
#include "dli.h"

void simulate(int width,
              int height,
              const thrust::device_vector<float> &in,
                    thrust::device_vector<float> &out,
              bool use_cub)
{
  if(use_cub)
  {
    cuda::std::mdspan temp_in(thrust::raw_pointer_cast(in.data()), height, width);
    cub::DeviceTransform::Transform(
        thrust::make_counting_iterator(0), out.begin(), width * height,
        [=] __host__ __device__(int id) { return dli::compute(id, temp_in); });
  }
  else
  {
    cuda::std::mdspan temp_in(thrust::raw_pointer_cast(in.data()), height, width);
    thrust::tabulate(out.begin(), out.end(), [=] __device__(int id) {
      return dli::compute(id, temp_in);
    });
  }
}

int main()
{
  int height = 2048;
  int width  = 8192;

  thrust::device_vector<float> d_prev = dli::init(height, width);
  thrust::device_vector<float> d_next(height * width);
  thrust::host_vector<float> h_prev(height * width);

  const int compute_steps = 500;
  const int write_steps = 3;
  for (int write_step = 0; write_step < write_steps; write_step++)
  {
    auto step_begin = std::chrono::high_resolution_clock::now();
    thrust::copy(d_prev.begin(), d_prev.end(), h_prev.begin());

    for (int compute_step = 0; compute_step < compute_steps; compute_step++)
    {
      simulate(width, height, d_prev, d_next, true);
      d_prev.swap(d_next);
    }

    auto write_begin = std::chrono::high_resolution_clock::now();
    dli::store(write_step, height, width, h_prev);
    auto write_end = std::chrono::high_resolution_clock::now();
    auto write_seconds = std::chrono::duration<double>(write_end - write_begin).count();

    auto step_end = std::chrono::high_resolution_clock::now();
    auto step_seconds = std::chrono::duration<double>(step_end - step_begin).count();
    std::printf("compute + write %d in %g s\n", write_step, step_seconds);
    std::printf("cub       write %d in %g s\n", write_step, write_seconds);
  }
  for (int write_step = 0; write_step < write_steps; write_step++) {
    auto step_begin = std::chrono::high_resolution_clock::now();
    thrust::copy(d_prev.begin(), d_prev.end(), h_prev.begin());

    for (int compute_step = 0; compute_step < compute_steps; compute_step++) {
      simulate(width, height, d_prev, d_next, false);
      d_prev.swap(d_next);
    }

    auto write_begin = std::chrono::high_resolution_clock::now();
    dli::store(write_step, height, width, h_prev);
    auto write_end = std::chrono::high_resolution_clock::now();
    auto write_seconds =
        std::chrono::duration<double>(write_end - write_begin).count();

    cudaDeviceSynchronize();
    auto step_end = std::chrono::high_resolution_clock::now();
    auto step_seconds =
        std::chrono::duration<double>(step_end - step_begin).count();
    std::printf("compute + write %d in %g s\n", write_step, step_seconds);
    std::printf("thrust    write %d in %g s\n", write_step, write_seconds);
  }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/compute-io-overlap.cu # build executable
!/tmp/a.out # run executable

In [ ]:
compute + write 0 in 0.0920168 s
cub       write 0 in 0.0833614 s
compute + write 1 in 0.247273 s
cub       write 1 in 0.0807166 s
compute + write 2 in 0.242608 s
cub       write 2 in 0.0803892 s
compute + write 0 in 0.481394 s
thrust    write 0 in 0.0816625 s
compute + write 1 in 0.327001 s
thrust    write 1 in 0.0808949 s
compute + write 2 in 0.32647 s
thrust    write 2 in 0.0802806 s